<a href="https://colab.research.google.com/github/TAUforPython/ts-forecasting-ensemble/blob/master/SMETS%20EEG%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# EEG Seizure Detection with SMETS - Google Colab Version
# Dataset: CHB-MIT Scalp EEG Database (PhysioNet)
# =============================================================================

# 1. УСТАНОВКА ЗАВИСИМОСТЕЙ
!pip install -q mne pywt tslearn pyedflib joblib matplotlib scikit-learn

ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt


In [ ]:
!pip install tslearn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.7/372.7 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install mne -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 49.1 MB/s eta 0:00:00


In [ ]:
!pip install PyWavelets -q

In [ ]:


# 2. ИМПОРТЫ
import numpy as np
import pandas as pd
import scipy.signal as signal
from scipy.stats import entropy
from scipy.ndimage import uniform_filter1d
from time import time  # ← ВАЖНО: было пропущено в оригинале
import re
import os
import glob
import logging
import gc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics import adjusted_rand_score
from joblib import Parallel, delayed
from tslearn.metrics import dtw
import mne
from mne.preprocessing import ICA
import pywt

In [ ]:
# 3. GOOGLE DRIVE & DATA SETUP
from google.colab import drive
drive.mount('/content/drive')

# Пути в Colab
BASE_DIR = '/content/drive/MyDrive/edf_process/EEG_SMETS'  # Измените на свой путь
DATA_DIR = '/content/chbmit_data'
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(BASE_DIR, exist_ok=True)

Mounted at /content/drive


# code 1

In [ ]:


# 4. ФУНКЦИИ ЗАГРУЗКИ ДАННЫХ С PHYSIONET
def download_chbmit_files(patient_ids, output_dir):
    """
    Загрузка EDF и summary-файлов с PhysioNet для указанных пациентов
    patient_ids: список строк ['01', '02', ...]
    """
    base_url = "https://physionet.org/files/chbmit/1.0.0"

    for pid in patient_ids:
        patient_dir = os.path.join(output_dir, f"chb{pid}")
        os.makedirs(patient_dir, exist_ok=True)

        # Скачиваем summary-файл
        summary_url = f"{base_url}/chb{pid}/chb{pid}-summary.txt"
        summary_path = os.path.join(patient_dir, f"chb{pid}-summary.txt")
        if not os.path.exists(summary_path):
            !wget -q -O {summary_path} {summary_url}
            print(f"✓ Скачан summary: chb{pid}-summary.txt")

        # Скачиваем все EDF-файлы пациента
        !wget -q -nH --cut-dirs=4 -P {patient_dir} -A "chb{pid}_*.edf" {base_url}/chb{pid}/
        print(f"✓ Скачаны EDF-файлы для chb{pid}")

def parse_chbmit_summary(summary_path, target_edf_filename):
    """
    Парсинг summary-файла CHB-MIT для извлечения интервалов приступов
    Возвращает список кортежей (start_sec, end_sec)
    """
    intervals = []
    try:
        with open(summary_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()

        # Разделяем по файлам
        file_blocks = re.split(r'File Name:\s*', content)

        for block in file_blocks:
            if target_edf_filename in block:
                lines = block.strip().split('\n')
                i = 0
                while i < len(lines):
                    line = lines[i].strip()
                    if 'Seizure' in line and 'Start Time' in line:
                        # Формат: "Seizure 1 Start Time: 2345 seconds"
                        match = re.search(r'(\d+(?:\.\d+)?)\s*seconds?', line)
                        if match:
                            start = float(match.group(1))
                            # Ищем End Time в следующих строках
                            for j in range(i+1, min(i+3, len(lines))):
                                end_match = re.search(r'(\d+(?:\.\d+)?)\s*seconds?', lines[j])
                                if end_match and 'End Time' in lines[j]:
                                    end = float(end_match.group(1))
                                    intervals.append((start, end))
                                    break
                    i += 1
                break
    except Exception as e:
        logger.error(f"Ошибка парсинга {summary_path}: {e}")

    return intervals

def get_all_edf_files(data_dir):
    """Поиск всех EDF-файлов в структуре CHB-MIT"""
    edf_files = []
    for root, dirs, files in os.walk(data_dir):
        for f in files:
            if f.endswith('.edf') and not f.endswith('.seizures'):
                edf_files.append(os.path.join(root, f))
    return edf_files

def classify_files(edf_files, data_dir):
    """Разделение файлов на seizure/normal на основе summary-файлов"""
    seizure_files = []
    normal_files = []

    for edf_path in edf_files:
        filename = os.path.basename(edf_path)
        patient_id = os.path.basename(os.path.dirname(edf_path))  # chbXX
        summary_path = os.path.join(data_dir, patient_id, f"{patient_id}-summary.txt")

        if os.path.exists(summary_path):
            intervals = parse_chbmit_summary(summary_path, filename)
            if intervals:
                seizure_files.append(edf_path)
            else:
                normal_files.append(edf_path)
        else:
            # Если summary нет - считаем normal (fallback)
            normal_files.append(edf_path)

    return normal_files, seizure_files

# 5. НАСТРОЙКА ЛОГИРОВАНИЯ
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(BASE_DIR, 'processing_log.txt')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# =============================================================================
# ОСНОВНЫЕ ФУНКЦИИ (без изменений логики, адаптированы пути)
# =============================================================================

def load_eeg(file_path):
    try:
        raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
        return raw
    except Exception as e:
        logger.error(f"Ошибка загрузки {file_path}: {e}")
        return None

def filter_artifacts(raw):
    if raw is None:
        return None
    raw.filter(l_freq=0.5, h_freq=70, method="iir")
    raw.notch_filter(freqs=50)

    ica = ICA(n_components=20, random_state=42, max_iter='auto')
    ica.fit(raw)
    ica_sources = ica.get_sources(raw).get_data()
    variances = np.var(ica_sources, axis=1)
    exclude = np.argsort(variances)[-2:]
    logger.info(f"Исключены ICA-компоненты: {exclude}")
    raw = ica.apply(raw, exclude=exclude)
    return raw

def normalize_data(raw):
    if raw is None:
        return None
    data = raw.get_data()
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)
    normalized_data = (data - mean) / (std + 1e-10)
    raw._data = normalized_data
    return raw

def segment_data(raw, window_size=1):
    if raw is None:
        return []
    sfreq = raw.info["sfreq"]
    window_samples = int(window_size * sfreq)
    data = raw.get_data()
    n_samples = data.shape[1]
    segments = []
    for start in range(0, n_samples, window_samples):
        end = start + window_samples
        if end <= n_samples:
            segment = data[:, start:end]
            segments.append(segment)
    logger.info(f"Сегментов: {len(segments)}, sfreq: {sfreq}, window: {window_samples}")
    return segments

def quality_control(segments):
    if not segments:
        return []
    clean_segments = []
    for i, seg in enumerate(segments):
        max_abs = np.max(np.abs(seg))
        std = np.std(seg)
        threshold = 10 * std
        if max_abs < threshold:
            clean_segments.append(seg)
        else:
            logger.info(f"Сегмент {i} отфильтрован: max={max_abs:.2f}, thr={threshold:.2f}")
    logger.info(f"QC: {len(segments)} → {len(clean_segments)} сегментов")
    return clean_segments

def dwt_transform(segment, level=5, top_k=47):
    if segment.size == 0:
        return np.array([])
    if segment.ndim == 1:
        segment = segment[np.newaxis, :]
    coeffs = pywt.wavedec(segment, 'db6', level=level, axis=-1)
    all_coeffs = np.concatenate([c[np.newaxis, :] if c.ndim == 1 else c for c in coeffs], axis=-1)
    if all_coeffs.ndim == 1:
        all_coeffs = all_coeffs[np.newaxis, :]
    top_indices = np.argsort(np.abs(all_coeffs), axis=-1)[:, -top_k:]
    top_coeffs = np.take_along_axis(all_coeffs, top_indices, axis=-1)
    return top_coeffs

def compute_distances(coeffs1, coeffs2):
    if coeffs1.ndim == 1: coeffs1 = coeffs1[np.newaxis, :]
    if coeffs2.ndim == 1: coeffs2 = coeffs2[np.newaxis, :]
    if coeffs1.shape != coeffs2.shape:
        raise ValueError(f"Shape mismatch: {coeffs1.shape} vs {coeffs2.shape}")
    return np.abs(coeffs1 - coeffs2)

def pair_matching(distances):
    if distances.size == 0:
        return []
    d = []
    dist_copy = distances.copy()
    while dist_copy.size > 0:
        min_dist = np.min(dist_copy)
        idx = np.unravel_index(np.argmin(dist_copy), dist_copy.shape)
        d.append(min_dist)
        dist_copy = np.delete(dist_copy, idx[0], axis=0)
        dist_copy = np.delete(dist_copy, idx[1], axis=1)
    return d

def p_norm(d, p):
    if not d:
        return 0
    return np.sum(np.abs(d) ** p) ** (1 / p)

def entropy_penalty(unmatched, coeffs1):
    EP = 0
    if unmatched and coeffs1.size > 0:
        for j in unmatched:
            hist = np.histogram(j, bins=10, density=True)[0]
            H_j = entropy(hist + 1e-10)  # avoid log(0)
            min_d_j = min([np.min(np.sqrt(np.sum((j - c) ** 2))) for c in coeffs1])
            denom = sum([entropy(np.histogram(c, bins=10, density=True)[0] + 1e-10) for c in coeffs1])
            RE_j = min_d_j * H_j / denom if denom != 0 else 0
            EP += RE_j
    return EP

def dimensionality_penalty(m, n):
    return (n - m) / (n + m) if (m + n) > 0 else 0

def SMETS(segment1, segment2):
    if segment1.size == 0 or segment2.size == 0:
        return 0, 0, 0
    coeffs1 = dwt_transform(segment1)
    coeffs2 = dwt_transform(segment2)
    if coeffs1.size == 0 or coeffs2.size == 0:
        return 0, 0, 0

    distances = compute_distances(coeffs1, coeffs2)
    d = pair_matching(distances)
    p = min(len(segment1), len(segment2))
    d_norm = p_norm(d, p)

    m, n = len(segment1), len(segment2)
    EP = 0
    if m < n:
        unmatched = [coeffs2[i] for i in range(m, n)]
        EP = entropy_penalty(unmatched, coeffs1)
    elif n < m:
        unmatched = [coeffs1[i] for i in range(n, m)]
        EP = entropy_penalty(unmatched, coeffs2)

    P = dimensionality_penalty(m, n)
    total_dist = np.sqrt((d_norm + 0.01 * EP) ** 2 + (0.01 * P)**2)
    return total_dist, P, EP

def compute_smets_to_reference(i, segment, reference_segment):
    total_dist, _, _ = SMETS(segment, reference_segment)
    return (i, total_dist)

def merge_intervals(intervals, max_gap=5):
    if not intervals:
        return []
    merged = []
    for start, end in sorted(intervals):
        if not merged or merged[-1][1] + max_gap < start:
            merged.append([start, end])
        else:
            merged[-1][1] = max(end, merged[-1][1])
    return merged

def filter_intervals(intervals, min_length=10):
    return [i for i in intervals if i[1] - i[0] >= min_length]

def extend_intervals(intervals, buffer=10, max_time=3600):
    extended = []
    for start, end in intervals:
        start = max(0, start - buffer)
        end = min(max_time, end + buffer)
        extended.append((start, end))
    return extended

def evaluate_intervals(detected, true_intervals, total_duration):
    if not true_intervals:
        sens = 1.0
        prec = 1.0 if not detected else 0.0
        f1 = 2 * sens * prec / (sens + prec) if (sens + prec) > 0 else 0.0
        return sens, prec, f1
    if not detected:
        return 0.0, 0.0, 0.0

    tp = fp = fn = 0
    for true_start, true_end in true_intervals:
        found = False
        for det_start, det_end in detected:
            overlap_start = max(true_start, det_start)
            overlap_end = min(true_end, det_end)
            if overlap_start < overlap_end:
                tp += overlap_end - overlap_start
                found = True
        if not found:
            fn += true_end - true_start

    for det_start, det_end in detected:
        overlap_found = False
        for true_start, true_end in true_intervals:
            if max(true_start, det_start) < min(true_end, det_end):
                overlap_found = True
                break
        if not overlap_found:
            fp += det_end - det_start

    sens = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    f1 = 2 * sens * prec / (sens + prec) if (sens + prec) > 0 else 0.0
    logger.info(f"TP={tp:.1f}, FP={fp:.1f}, FN={fn:.1f} | Sens={sens:.3f}, Prec={prec:.3f}, F1={f1:.3f}")
    return sens, prec, f1

def detect_seizure_with_smets(segments, window_size, sfreq, max_duration, true_intervals, n_jobs=4):
    start_time = time()

    # Выбор эталона (нормальные сегменты)
    n_ref = min(100, len(segments))
    ref_candidates = []
    for i in range(n_ref):
        seg_start = i * window_size
        seg_end = (i + 1) * window_size
        is_seizure = any(max(start, seg_start) < min(end, seg_end) for start, end in true_intervals)
        if not is_seizure:
            ref_candidates.append(segments[i])

    if not ref_candidates:
        ref_candidates = segments[:n_ref]
        logger.warning("Нет нормальных сегментов для эталона, используем первые 100")

    amplitudes = [np.max(np.abs(seg)) for seg in ref_candidates]
    q25 = np.percentile(amplitudes, 25)
    ref_segments = [seg for seg, amp in zip(ref_candidates, amplitudes) if amp < q25]
    reference_data = np.mean(ref_segments, axis=0) if ref_segments else segments[0]

    # Параллельное вычисление SMETS
    logger.info(f"Вычисление SMETS для {len(segments)} сегментов...")
    results = Parallel(n_jobs=n_jobs, verbose=0)(
        delayed(compute_smets_to_reference)(i, segment, reference_data)
        for i, segment in enumerate(segments)
    )

    smets_distances = sorted(results, key=lambda x: x[0])
    smets_distances = [(i * window_size, dist) for i, dist in smets_distances]

    # Сглаживание
    distances = [d for _, d in smets_distances]
    smoothed = uniform_filter1d(distances, size=3)
    smets_distances = [(t, d) for (t, _), d in zip(smets_distances, smoothed)]

    # Адаптивный порог IQR
    q75, q25 = np.percentile(smoothed, [75, 25])
    iqr = q75 - q25
    threshold = max(q75 + 2.5 * iqr, 250)
    logger.info(f"Порог: {threshold:.2f} (Q75={q75:.2f}, IQR={iqr:.2f})")

    # Выделение интервалов
    seizure_intervals = []
    start = None
    for (idx, _), dist in zip(smets_distances, smoothed):
        if dist > threshold and start is None:
            start = idx
        elif dist <= threshold and start is not None:
            if idx - start >= 10:
                seizure_intervals.append((start, idx))
            start = None
    if start is not None and len(segments) * window_size - start >= 10:
        seizure_intervals.append((start, len(segments) * window_size))

    # Пост-обработка
    seizure_intervals = merge_intervals(seizure_intervals, max_gap=5)
    seizure_intervals = filter_intervals(seizure_intervals, min_length=10)
    seizure_intervals = extend_intervals(seizure_intervals, buffer=10, max_time=max_duration)

    logger.info(f"SMETS время: {time() - start_time:.2f}с, обнаружено интервалов: {len(seizure_intervals)}")
    return smets_distances, [threshold] * len(smets_distances), seizure_intervals

def process_file(file_path, base_path, window_size, n_jobs):
    start_time = time()
    file_name = os.path.basename(file_path)
    logger.info(f"→ Обработка: {file_name}")

    raw = load_eeg(file_path)
    if raw is None:
        return 0.0, 0.0, 0.0, time() - start_time, file_name

    max_duration = raw.n_times / raw.info['sfreq']
    raw = filter_artifacts(raw)
    raw = normalize_data(raw)
    segments = segment_data(raw, window_size)
    segments = quality_control(segments)

    # Поиск true intervals
    patient_id = os.path.basename(os.path.dirname(file_path))
    summary_path = os.path.join(base_path, patient_id, f"{patient_id}-summary.txt")
    true_intervals = parse_chbmit_summary(summary_path, file_name) if os.path.exists(summary_path) else []

    # Детекция
    smets_dist, threshold, detected = detect_seizure_with_smets(
        segments, window_size, raw.info['sfreq'], max_duration, true_intervals, n_jobs
    )

    # Оценка
    sens, prec, f1 = evaluate_intervals(detected, true_intervals, max_duration)

    # Визуализация (сохранение в Google Drive)
    times = [t for t, _ in smets_dist]
    dists = [d for _, d in smets_dist]

    plt.figure(figsize=(14, 5))
    plt.plot(times, dists, label='SMETS distance', linewidth=1)
    plt.plot(times, threshold, 'r--', label='Adaptive threshold', alpha=0.7)

    for s, e in detected:
        plt.axvspan(s, e, color='orange', alpha=0.3, label='Detected' if s == (detected[0][0] if detected else 0) else "")
    for s, e in true_intervals:
        plt.axvspan(s, e, color='green', alpha=0.2, label='Ground truth' if s == (true_intervals[0][0] if true_intervals else 0) else "")

    plt.xlabel('Time (s)')
    plt.ylabel('SMETS distance')
    plt.title(f'Seizure Detection: {file_name}')
    plt.legend(fontsize=8)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    plot_path = os.path.join(BASE_DIR, f'plot_{file_name}.png')
    plt.savefig(plot_path, dpi=150)
    plt.close()

    # Лог результатов
    with open(os.path.join(BASE_DIR, 'results_summary.txt'), 'a') as f:
        f.write(f"{file_name}\t{sens:.3f}\t{prec:.3f}\t{f1:.3f}\t{time()-start_time:.1f}s\n")

    gc.collect()
    return sens, prec, f1, time() - start_time, file_name

def process_chbmit_data(base_path, window_size=1, n_jobs=4, max_patients=3, max_files_per_patient=2):
    """
    Основная функция обработки CHB-MIT dataset в Colab
    """
    total_start = time()
    logger.info(f"🚀 Старт обработки | patients≤{max_patients}, files/patient≤{max_files_per_patient}")

    # Скачиваем данные (пример: пациенты 01-03)
    patient_ids = [f"{i:02d}" for i in range(1, max_patients+1)]
    download_chbmit_files(patient_ids, DATA_DIR)

    # Поиск и классификация файлов
    all_edf = get_all_edf_files(DATA_DIR)
    normal_files, seizure_files = classify_files(all_edf, DATA_DIR)

    logger.info(f"📁 Найдено: {len(normal_files)} normal, {len(seizure_files)} seizure файлов")

    # Ограничение количества
    seizure_files = seizure_files[:max_files_per_patient * len(patient_ids)]
    normal_files = normal_files[:max_files_per_patient * len(patient_ids)]

    files_to_process = seizure_files + normal_files
    logger.info(f"⚙️  Обрабатываем {len(files_to_process)} файлов")

    # Обработка
    results = []
    for i, fpath in enumerate(files_to_process, 1):
        logger.info(f"[{i}/{len(files_to_process)}] {os.path.basename(fpath)}")
        sens, prec, f1, t, name = process_file(fpath, DATA_DIR, window_size, n_jobs)
        results.append((name, sens, prec, f1, t))
        gc.collect()

    # Итоговый отчёт
    report_path = os.path.join(BASE_DIR, 'final_report.txt')
    with open(report_path, 'w') as f:
        f.write("CHB-MIT Seizure Detection Results (SMETS)\n")
        f.write("="*60 + "\n")
        f.write(f"{'File':<30} {'Sens':>8} {'Prec':>8} {'F1':>8} {'Time':>10}\n")
        f.write("-"*60 + "\n")
        for name, s, p, f1, t in results:
            f.write(f"{name:<30} {s:>8.3f} {p:>8.3f} {f1:>8.3f} {t:>9.1f}s\n")
        f.write("-"*60 + "\n")

        # Средние метрики
        if results:
            avg_sens = np.mean([r[1] for r in results])
            avg_prec = np.mean([r[2] for r in results])
            avg_f1 = np.mean([r[3] for r in results])
            f.write(f"\n{'AVERAGE':<30} {avg_sens:>8.3f} {avg_prec:>8.3f} {avg_f1:>8.3f}\n")
            f.write(f"Total time: {time() - total_start:.1f} seconds\n")

    logger.info(f"✅ Готово! Отчёт: {report_path}")
    logger.info(f"📊 Графики сохранены в: {BASE_DIR}")

    # Очистка временных данных (опционально)
    # !rm -rf /content/chbmit_data

    return results

# =============================================================================
# ЗАПУСК
# =============================================================================
if __name__ == "__main__":
    # Настройки
    WINDOW_SIZE = 1  # секунды
    N_JOBS = 2       # Colab: не ставьте >2-4 из-за лимитов CPU
    MAX_PATIENTS = 2 # начните с малого для теста
    MAX_FILES = 1    # файлов на пациента

    print("🔧 Инициализация...")
    results = process_chbmit_data(
        base_path=DATA_DIR,
        window_size=WINDOW_SIZE,
        n_jobs=N_JOBS,
        max_patients=MAX_PATIENTS,
        max_files_per_patient=MAX_FILES
    )

    # Быстрый просмотр результатов
    if results:
        print("\n📈 Результаты:")
        for name, s, p, f1, t in results:
            print(f"{name:25s} | F1: {f1:.3f} | Time: {t:.1f}s")

🔧 Инициализация...
✓ Скачан summary: chb01-summary.txt
✓ Скачаны EDF-файлы для chb01
✓ Скачан summary: chb02-summary.txt
✓ Скачаны EDF-файлы для chb02


# code 2

In [ ]:
import numpy as np
import pandas as pd
import scipy.signal as signal
from scipy.stats import entropy
from scipy.ndimage import uniform_filter1d
import pywt
import os
import re
import glob
import matplotlib.pyplot as plt
import gc
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics import adjusted_rand_score
import logging
from joblib import Parallel, delayed
from tslearn.metrics import dtw
import pyedflib
from time import time
import shutil

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set working directory on Google Drive
os.chdir('/content/drive/MyDrive/edf_process/EEG_SMETS')

## Configuration and Logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - INFO - %(message)s')
logger = logging.getLogger(__name__)

# Log to file on Google Drive
file_handler = logging.FileHandler('/content/drive/MyDrive/edf_process/EEG_SMETS/generalized_processing_log.txt')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - INFO - %(message)s'))
logger.addHandler(file_handler)

## Data Loading Functions (from example)

def read_edf(file_path):
    """Read EDF file using pyedflib"""
    with pyedflib.EdfReader(file_path) as f:
        n = f.signals_in_file
        signal_labels = f.getSignalLabels()
        signals = np.zeros((n, f.getNSamples()[0]))
        for i in range(n):
            signals[i, :] = f.readSignal(i)
    return signals, signal_labels

def parse_summary_file(summary_file_path):
    """Parse PhysioNet summary file to extract seizure information"""
    edf_files = []
    with open(summary_file_path, 'r') as file:
        content = file.read()
        files = re.split(r'File Name: ', content)[1:]  # Split by files
        for file_info in files:
            lines = file_info.split('\n')
            file_name = lines[0].strip()
            seizure_start = []
            seizure_end = []
            num_seizures = 0
            k = 0
            for line in lines[1:]:
                if 'Number of Seizures in File' in line:
                    num_seizures = int(line.split(':')[-1].strip())
                    k = num_seizures
                    continue

                if k > 0:
                    if ('Seizure' in line) and ('Start Time' in line):
                        seizure_start_i = int(re.search(r'\d+', line.split(':')[-1].strip()).group())
                        seizure_start.append(seizure_start_i)
                        continue

                    if ('Seizure' in line) and ('End Time' in line):
                        seizure_end_i = int(re.search(r'\d+', line.split(':')[-1].strip()).group())
                        seizure_end.append(seizure_end_i)
                        k -= 1
                        continue

            edf_files.append({
                'file_name': file_name,
                'seizure_start': seizure_start,
                'seizure_end': seizure_end,
                'num_seizures': num_seizures
            })
    return edf_files

## Signal Processing Functions

def butter_bandpass_filter(data, fs, order=5):
    """Butterworth bandpass filter for EEG frequency bands"""
    # Frequency bands: Delta, Theta, Alpha, Beta, Gamma
    lowcut = [1, 4, 8, 14, 30]
    highcut = [4, 8, 14, 30, 50]
    nyquist = 0.5 * fs
    filtered_signals = []

    for i in range(len(lowcut)):
        low = lowcut[i] / nyquist
        high = highcut[i] / nyquist
        b, a = signal.butter(order, [low, high], btype='band')
        filtered_data = signal.lfilter(b, a, data)
        filtered_signals.append(filtered_data)

    return filtered_signals

def measured(signal, window_size, sampling_time):
    """Remove DC offset using moving window"""
    window_size = int(window_size * sampling_time)
    n = signal.size // window_size
    f_signal = np.copy(signal)
    for i in range(int(n)):
        a = i * window_size
        b = (i + 1) * window_size
        f_signal[a:b] = signal[a:b] - np.mean(signal[a:b])
    return f_signal

def mean_delete(filtered_signals, window_count, sampling_rate):
    """Remove mean from filtered signals"""
    processed_signals = []
    for filtered in filtered_signals:
        filtered_mean = measured(filtered, window_count, sampling_rate)
        processed_signals.append(filtered_mean)
    return processed_signals

## EEG Processing Functions (adapted for numpy arrays)

def load_eeg_from_array(signals, sampling_rate=256):
    """Convert numpy array to mne-like structure"""
    info = {
        'sfreq': sampling_rate,
        'n_times': signals.shape[1],
        'ch_names': [f'CH{i}' for i in range(signals.shape[0])]
    }
    return signals, info

def filter_artifacts(signals, sampling_rate=256):
    """Filter artifacts from EEG signals"""
    if signals is None:
        return None

    # Apply bandpass filter (0.5-70 Hz)
    nyquist = 0.5 * sampling_rate
    low = 0.5 / nyquist
    high = 70.0 / nyquist
    b, a = signal.butter(5, [low, high], btype='band')
    filtered = signal.lfilter(b, a, signals, axis=1)

    # Notch filter at 50 Hz
    b_notch, a_notch = signal.iirnotch(50.0, 30.0, sampling_rate)
    filtered = signal.lfilter(b_notch, a_notch, filtered, axis=1)

    return filtered

def normalize_data(signals):
    """Z-score normalization"""
    if signals is None:
        return None
    mean = np.mean(signals, axis=1, keepdims=True)
    std = np.std(signals, axis=1, keepdims=True)
    normalized = (signals - mean) / (std + 1e-10)
    return normalized

def segment_data(signals, sampling_rate, window_size=1):
    """Segment data into windows"""
    if signals is None:
        return []

    window_samples = int(window_size * sampling_rate)
    n_samples = signals.shape[1]
    segments = []

    for start in range(0, n_samples, window_samples):
        end = start + window_samples
        if end <= n_samples:
            segment = signals[:, start:end]
            segments.append(segment)

    logger.info(f"Created segments: {len(segments)}, sampling rate: {sampling_rate}, window: {window_samples} samples")
    return segments

def quality_control(segments):
    """Filter out noisy segments"""
    if not segments:
        return []

    clean_segments = []
    for i, seg in enumerate(segments):
        max_abs = np.max(np.abs(seg))
        std = np.std(seg)
        threshold = 10 * std
        if max_abs < threshold:
            clean_segments.append(seg)
        else:
            logger.info(f"Segment {i} filtered: max_abs={max_abs:.2f}, threshold={threshold:.2f}")

    logger.info(f"Total segments: {len(segments)}, Clean segments: {len(clean_segments)}")
    return clean_segments

## SMETS Functions (unchanged logic)

def dwt_transform(segment, level=5, top_k=47):
    """Discrete Wavelet Transform"""
    if segment.size == 0:
        return np.array([])
    if segment.ndim == 1:
        segment = segment[np.newaxis, :]

    coeffs = pywt.wavedec(segment, 'db6', level=level, axis=-1)
    all_coeffs = np.concatenate([c[np.newaxis, :] if c.ndim == 1 else c for c in coeffs], axis=-1)

    if all_coeffs.ndim == 1:
        all_coeffs = all_coeffs[np.newaxis, :]

    top_indices = np.argsort(np.abs(all_coeffs), axis=-1)[:, -top_k:]
    top_coeffs = np.take_along_axis(all_coeffs, top_indices, axis=-1)
    return top_coeffs

def compute_distances(coeffs1, coeffs2):
    """Compute distances between coefficients"""
    if coeffs1.ndim == 1:
        coeffs1 = coeffs1[np.newaxis, :]
    if coeffs2.ndim == 1:
        coeffs2 = coeffs2[np.newaxis, :]
    if coeffs1.shape != coeffs2.shape:
        raise ValueError(f"Shape mismatch: coeffs1={coeffs1.shape}, coeffs2={coeffs2.shape}")
    return np.abs(coeffs1 - coeffs2)

def pair_matching(distances):
    """Pair matching algorithm"""
    if distances.size == 0:
        return []

    d = []
    distances = distances.copy()
    while distances.size > 0:
        min_dist = np.min(distances)
        idx = np.unravel_index(np.argmin(distances), distances.shape)
        d.append(min_dist)
        distances = np.delete(distances, idx[0], axis=0)
        distances = np.delete(distances, idx[1], axis=1)
    return d

def p_norm(d, p):
    """p-norm calculation"""
    if not d:
        return 0
    norm = np.sum(np.abs(d) ** p) ** (1 / p)
    return norm

def entropy_penalty(unmatched, coeffs1):
    """Entropy penalty for unmatched components"""
    EP = 0
    if unmatched and coeffs1.size > 0:
        for j in unmatched:
            H_j = entropy(np.histogram(j, bins=10, density=True)[0])
            min_d_j = min([np.min(np.sqrt(np.sum((j - c) ** 2))) for c in coeffs1])
            denom = sum([entropy(np.histogram(c, bins=10, density=True)[0]) for c in coeffs1])
            RE_j = min_d_j * H_j / denom if denom != 0 else 0
            EP += RE_j
    return EP

def dimensionality_penalty(m, n):
    """Penalty for dimensionality difference"""
    return (n - m) / (n + m) if (m + n) > 0 else 0

def SMETS(segment1, segment2):
    """SMETS distance calculation"""
    if segment1.size == 0 or segment2.size == 0:
        return 0, 0, 0

    coeffs1 = dwt_transform(segment1)
    coeffs2 = dwt_transform(segment2)

    if coeffs1.size == 0 or coeffs2.size == 0:
        return 0, 0, 0

    distances = compute_distances(coeffs1, coeffs2)
    d = pair_matching(distances)
    p = min(len(segment1), len(segment2))
    d_norm = p_norm(d, p)

    unmatched = []
    m, n = len(segment1), len(segment2)

    if m < n:
        unmatched = [coeffs2[i] for i in range(m, n)]
        EP = entropy_penalty(unmatched, coeffs1)
    elif n < m:
        unmatched = [coeffs1[i] for i in range(n, m)]
        EP = entropy_penalty(unmatched, coeffs2)
    else:
        EP = 0

    P = dimensionality_penalty(m, n)
    total_dist = np.sqrt((d_norm + 0.01 * EP) ** 2 + (0.01 * P) ** 2)

    logger.info(f"SMETS: d_norm={d_norm}, EP={EP}, P={P}, total_dist={total_dist}")
    return total_dist, P, EP

def compute_smets_to_reference(i, segment, reference_segment):
    """Compute SMETS distance to reference"""
    total_dist, _, _ = SMETS(segment, reference_segment)
    return (i, total_dist)

## Interval Processing Functions

def get_seizure_intervals_from_parsed(edf_file_info):
    """Get seizure intervals from parsed summary data"""
    intervals = []
    file_name = edf_file_info['file_name']
    logger.info(f"Processing intervals for: {file_name}")

    for start, end in zip(edf_file_info['seizure_start'], edf_file_info['seizure_end']):
        intervals.append((float(start), float(end)))
        logger.info(f"Seizure interval: {start} - {end}")

    logger.info(f"Seizure intervals for {file_name}: {intervals}")
    return intervals

def merge_intervals(intervals, max_gap=5):
    """Merge close intervals"""
    if not intervals:
        return []
    merged = []
    for start, end in sorted(intervals):
        if not merged or merged[-1][1] + max_gap < start:
            merged.append([start, end])
        else:
            merged[-1][1] = max(end, merged[-1][1])
    return merged

def filter_intervals(intervals, min_length=10):
    """Filter short intervals"""
    return [i for i in intervals if i[1] - i[0] >= min_length]

def extend_intervals(intervals, buffer=10, max_time=3600):
    """Extend intervals with buffer"""
    extended = []
    for start, end in intervals:
        start = max(0, start - buffer)
        end = min(max_time, end + buffer)
        extended.append((start, end))
    return extended

## Evaluation Functions

def evaluate_intervals(detected, true_intervals, total_duration):
    """Evaluate detection performance"""
    logger.info(f"Evaluating intervals: detected={detected}, true={true_intervals}, duration={total_duration}")

    if not true_intervals:
        sensitivity = 1.0
        precision = 1.0 if not detected else 0.0
        f1 = 2 * (sensitivity * precision) / (sensitivity + precision) if (sensitivity + precision) > 0 else 0.0
        logger.info(f"No true seizures: sensitivity={sensitivity}, precision={precision}, F1={f1}")
        return sensitivity, precision, f1

    if not detected:
        logger.info("No detected seizures: sensitivity=0, precision=0, F1=0")
        return 0.0, 0.0, 0.0

    tp = 0
    fp = 0
    fn = 0

    for true_start, true_end in true_intervals:
        found = False
        for det_start, det_end in detected:
            overlap_start = max(true_start, det_start)
            overlap_end = min(true_end, det_end)
            if overlap_start < overlap_end:
                tp += overlap_end - overlap_start
                found = True
        if not found:
            fn += true_end - true_start

    for det_start, det_end in detected:
        overlap_found = False
        for true_start, true_end in true_intervals:
            overlap_start = max(true_start, det_start)
            overlap_end = min(true_end, det_end)
            if overlap_start < overlap_end:
                overlap_found = True
                break
        if not overlap_found:
            fp += det_end - det_start

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    f1 = 2 * (sensitivity * precision) / (sensitivity + precision) if (sensitivity + precision) > 0 else 0.0

    logger.info(f"TP={tp}, FP={fp}, FN={fn}, sensitivity={sensitivity}, precision={precision}, F1={f1}")
    return sensitivity, precision, f1

## Detection Function

def detect_seizure_with_smets(segments, window_size, sfreq, max_duration, true_intervals, n_jobs=16):
    """Detect seizures using SMETS"""
    start_time = time()

    # Select reference normal segment
    n_reference = min(100, len(segments))
    reference_candidates = []

    for i in range(n_reference):
        segment_start = i * window_size
        segment_end = (i + 1) * window_size
        is_seizure = any(max(start, segment_start) < min(end, segment_end) for start, end in true_intervals)
        if not is_seizure:
            reference_candidates.append(segments[i])

    if not reference_candidates:
        reference_candidates = segments[:n_reference]
        logger.warning("Could not find non-seizure segments for reference, using first 100 segments")

    amplitudes = [np.max(np.abs(seg)) for seg in reference_candidates]
    q25 = np.percentile(amplitudes, 25)
    reference_segments = [seg for seg, amp in zip(reference_candidates, amplitudes) if amp < q25]
    reference_data = np.mean(reference_segments, axis=0) if reference_segments else segments[0]
    logger.info(f"Selected {len(reference_segments)} reference segments for averaging")

    # Parallel SMETS computation
    smets_start = time()
    results = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(compute_smets_to_reference)(i, segment, reference_data) for i, segment in enumerate(segments)
    )
    smets_time = time() - smets_start
    logger.info(f"SMETS computation time: {smets_time:.2f} sec")

    smets_distances = sorted(results, key=lambda x: x[0])
    smets_distances = [(i * window_size, dist) for i, dist in smets_distances]

    # Smooth distances
    distances = [d for _, d in smets_distances]
    smoothed_distances = uniform_filter1d(distances, size=3)
    smets_distances = [(t, d) for (t, _), d in zip(smets_distances, smoothed_distances)]

    # Adaptive threshold based on IQR
    q75 = np.percentile(smoothed_distances, 75)
    q25 = np.percentile(smoothed_distances, 25)
    iqr = q75 - q25
    threshold = max(q75 + 2.5 * iqr, 250)
    threshold_list = [threshold] * len(smets_distances)
    logger.info(f"Adaptive SMETS threshold: {threshold:.2f} (Q75={q75}, IQR={iqr})")

    # Log max SMETS in seizure region
    max_index = len(smoothed_distances) - 1
    if true_intervals:
        start = min(t[0] for t in true_intervals)
        end = max(t[1] for t in true_intervals)
        seizure_range = range(max(0, int(start / window_size)), min(max_index + 1, int(end / window_size) + 1))
    else:
        seizure_range = range(max(0, len(smets_distances) - 50), min(max_index + 1, len(smets_distances)))

    max_smets_in_seizure = max((smoothed_distances[i] for i in seizure_range if i <= max_index), default=0)
    exceed_points = [(i * window_size, dist) for i, (t, dist) in enumerate(smets_distances) if dist > threshold]
    logger.info(f"Max SMETS near seizure: {max_smets_in_seizure:.2f}")
    logger.info(f"SMETS threshold exceed points: {exceed_points}")

    # Extract seizure intervals
    seizure_intervals = []
    start = None
    for (idx, _), dist in zip(smets_distances, smoothed_distances):
        if dist > threshold and start is None:
            start = idx
        elif dist <= threshold and start is not None:
            if idx - start >= 10:
                seizure_intervals.append((start, idx))
            start = None

    if start is not None and len(segments) * window_size - start >= 10:
        seizure_intervals.append((start, len(segments) * window_size))

    # Merge, filter and extend intervals
    logger.info(f"Intermediate intervals before extension: {seizure_intervals}")
    seizure_intervals = merge_intervals(seizure_intervals, max_gap=5)
    seizure_intervals = filter_intervals(seizure_intervals, min_length=10)
    seizure_intervals = extend_intervals(seizure_intervals, buffer=10, max_time=max_duration)
    logger.info(f"Intervals after extension: {seizure_intervals}")

    logger.info(f"Total processing time in detect_seizure_with_smets: {time() - start_time:.2f} sec")
    return smets_distances, threshold_list, seizure_intervals

## Main Processing Function

def process_file_from_physionet(edf_file_info, patient_id, base_temp_path, window_size, n_jobs, sampling_rate=256):
    """Process a single EDF file from PhysioNet"""
    start_time = time()
    file_name = edf_file_info['file_name']
    logger.info(f"Processing file: {file_name} for patient {patient_id}")

    # Download EDF file from PhysioNet
    os.chdir(base_temp_path)
    edf_directory = f'/physionet.org/files/chbmit/1.0.0/{patient_id}/'
    edf_file_link = f"https:/}{edf_directory}{file_name}"

    try:
        !wget -r -c -N -np {edf_file_link}
    except Exception as e:
        logger.error(f"Error downloading {file_name}: {e}")
        return 0.0, 0.0, 0.0, time() - start_time, file_name

    edf_file_path = f"{base_temp_path}{edf_directory}{file_name}"

    if not os.path.exists(edf_file_path):
        logger.error(f"Downloaded file not found: {edf_file_path}")
        return 0.0, 0.0, 0.0, time() - start_time, file_name

    # Read EDF file
    try:
        signals, labels = read_edf(edf_file_path)
    except Exception as e:
        logger.error(f"Error reading {edf_file_path}: {e}")
        return 0.0, 0.0, 0.0, time() - start_time, file_name

    max_duration = signals.shape[1] / sampling_rate
    logger.info(f"Recording duration: {max_duration} seconds")

    # Process signals
    signals = filter_artifacts(signals, sampling_rate)
    signals = normalize_data(signals)
    segments = segment_data(signals, sampling_rate, window_size)
    segments = quality_control(segments)
    logger.info(f"Number of segments after quality control: {len(segments)}")

    # Get seizure intervals from parsed data
    true_intervals = []
    if edf_file_info['num_seizures'] > 0:
        true_intervals = get_seizure_intervals_from_parsed(edf_file_info)

    # Detect seizures
    smets_distances, threshold, detected_intervals = detect_seizure_with_smets(
        segments, window_size, sampling_rate, max_duration, true_intervals, n_jobs
    )

    logger.info(f"Detected seizure intervals: {detected_intervals}")
    logger.info(f"True seizure intervals: {true_intervals}")

    # Evaluate
    sensitivity, precision, f1 = evaluate_intervals(detected_intervals, true_intervals, max_duration)

    # Visualization
    times = [t for t, _ in smets_distances]
    distances = [d for _, d in smets_distances]

    plt.figure(figsize=(12, 6))
    plt.plot(times, distances, label='SMETS distance')
    plt.plot(times, threshold, 'r--', label='Adaptive threshold')

    for start, end in detected_intervals:
        plt.axvspan(start, end, color='yellow', alpha=0.3, label='Detected seizure' if start == detected_intervals[0][0] else "")

    for start, end in true_intervals:
        plt.axvspan(start, end, color='green', alpha=0.2, label='True seizure' if start == true_intervals[0][0] else "")

    plt.xlabel('Time (s)')
    plt.ylabel('SMETS distance')
    plt.title(f'Seizure Detection using SMETS ({file_name})')
    plt.legend()

    # Save to Google Drive
    output_path = f'/content/drive/MyDrive/edf_process/EEG_SMETS/smets_plot_{file_name}.png'
    plt.savefig(output_path)
    plt.close()

    # Log results
    with open('/content/drive/MyDrive/edf_process/EEG_SMETS/generalized_processing_log.txt', 'a') as f:
        f.write(f"File: {file_name}\n")
        f.write(f"Detected intervals: {detected_intervals}\n")
        f.write(f"True intervals: {true_intervals}\n")
        f.write(f"Sensitivity: {sensitivity:.2f}, Precision: {precision:.2f}, F1: {f1:.2f}\n")
        f.write(f"Processing time: {time() - start_time:.2f} seconds\n\n")

    # Cleanup temporary files
    try:
        shutil.rmtree(f'{base_temp_path}/physionet.org')
    except:
        pass

    return sensitivity, precision, f1, time() - start_time, file_name

## Main Execution

def process_chb_mit_data_colab(patient_ids=None, window_size=1, n_jobs=16, max_files_per_type=2):
    """Process CHB-MIT data from PhysioNet in Google Colab"""
    total_start_time = time()

    # Setup paths
    base_temp_path = '/content'
    output_base = '/content/drive/MyDrive/edf_process/EEG_SMETS'
    os.makedirs(output_base, exist_ok=True)

    if patient_ids is None:
        patient_ids = ['chb01']  # Default to chb01

    all_results = []

    for patient_id in patient_ids:
        logger.info(f"Processing patient: {patient_id}")

        # Download summary file
        os.chdir(base_temp_path)
        summary_file_path = f'/physionet.org/files/chbmit/1.0.0/{patient_id}/{patient_id}-summary.txt'
        physionet_summary_url = f'https:/{summary_file_path}'

        try:
            !wget -r -c -N --no-parent -np {physionet_summary_url}
        except Exception as e:
            logger.error(f"Error downloading summary for {patient_id}: {e}")
            continue

        # Parse summary
        full_summary_path = f'{base_temp_path}{summary_file_path}'
        if not os.path.exists(full_summary_path):
            logger.error(f"Summary file not found: {full_summary_path}")
            continue

        edf_files_list = parse_summary_file(full_summary_path)

        # Separate seizure and normal files
        seizure_files = [f for f in edf_files_list if f['num_seizures'] > 0]
        normal_files = [f for f in edf_files_list if f['num_seizures'] == 0]

        logger.info(f"Patient {patient_id}: {len(seizure_files)} seizure files, {len(normal_files)} normal files")

        # Limit files if specified
        if max_files_per_type:
            seizure_files = seizure_files[:max_files_per_type]
            normal_files = normal_files[:max_files_per_type]

        files_to_process = seizure_files + normal_files

        # Process files
        for edf_file in files_to_process:
            sens, prec, f1, proc_time, file_name = process_file_from_physionet(
                edf_file, patient_id, base_temp_path, window_size, n_jobs
            )
            all_results.append((file_name, sens, prec, f1, proc_time))

    # Save summary
    summary_path = f'{output_base}/generalized_processing_summary.txt'
    with open(summary_path, 'w') as f:
        f.write("Final Results Table\n")
        f.write("==================\n")
        f.write("File\tSensitivity\tPrecision\tF1\tProcessing Time (s)\n")
        for file_name, sens, prec, f1, t in all_results:
            f.write(f"{file_name}\t{sens:.2f}\t\t{prec:.2f}\t\t{f1:.2f}\t\t{t:.2f}\n")
        f.write(f"\nTotal processing time: {time() - total_start_time:.2f} seconds\n")

    logger.info(f"Total processing time for all files: {time() - total_start_time:.2f} seconds")
    return all_results

# Run processing
if __name__ == "__main__":
    # Process patients chb01, chb02, chb03 (example)
    patient_ids = ['chb01', 'chb02', 'chb03']
    process_chb_mit_data_colab(patient_ids=patient_ids, window_size=1, n_jobs=16, max_files_per_type=2)

# code 3